In [1]:
input_x = '/home/snu/Downloads/자료 정리/X_raw.txt'
input_y = '/home/snu/Downloads/자료 정리/y_raw.txt'

image_x = 75
image_y = 75
image_z = 2

No_data = 0

y_class = 2

Jump = 0.00001

Batch_X = 30
Batch_Y = 101

structure ='''

#<<< Structure >>>
#v3
# Convolution
conv(1,2,8)
conv(2,8,32, relu=0)
conv(3,32,32)
conv(4,32,64,relu=0)
conv(5,32,64,-2); max_p(2)
conv(6,64,64,relu=0)
conv(7,64,64,2)
conv(8,64,64,relu=0)
conv(9,64,64,2)
conv(10,64,128,relu=0)
conv(11,64,128,-2); max_p(2)
conv(12,128,128,relu=0)
conv(13,128,128,2)
conv(14,128,128,relu=0)
conv(15,128,128,2)
conv(16,128,256,relu=0)
conv(17,128,256,-2); max_p(2)
conv(18,256,256,relu=0)
conv(19,256,256,2)
conv(20,256,256,relu=0)
conv(21,256,256,2)
conv(22,256,512,relu=0)
conv(23,256,512,-2); max_p(2)
conv(24,512,512,relu=0)
conv(25,512,512,2)
conv(26,512,512,relu=0)
conv(27,512,512,2)
conv(28,512,1024,relu=0)
conv(29,512,1024,-2)

# Fully connected 
full(30,1024,64)
'''

In [ ]:
a=[1,2,3,4,5,6,7,8,9,10,11]

for idx in range(0, a.shape) 


def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert inputs.shape[0] == targets.shape[0]
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
for idx in range(0, a.shape[0]+1, batch_size):
    print(a[idx])
    result = a[idx:idx+batch_size]
    print(result)

In [2]:
import pickle
import numpy as np
import tensorflow as tf
import email_sending
from PIL import Image, ImageDraw

In [5]:
def unpickle(file):
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data

In [6]:
unpickle(input_x)

UnpicklingError: invalid load key, '['.

In [ ]:
#load the CIFAR-10
def load_CIFAR10(pos, n_chunks=1):
    Xtr = []
    Ytr = []
    for i in range(n_chunks):
        train = unpickle(pos + '/data_batch_{0}'.format(i + 1))
        Xtr.extend(train[b'data'])
        Ytr.extend(train[b'labels'])
        test = unpickle(pos + '/test_batch')
        Xte = test[b'data'] 
        Yte = test[b'labels']
    return np.array(Xtr), np.array(Ytr), np.array(Xte), np.array(Yte)

In [ ]:
# expresses the label data in one-hot encoding.
def onehot_encoding (Ytr, Yte):
    Ytr_onehot = np.zeros((Ytr.size, int(y_class)))
    Yte_onehot = np.zeros((Yte.size, int(y_class)))
    for i in range(Ytr.size):
        Ytr_onehot[i][Ytr[i]] = 1
    for i in range(Yte.size):
        Yte_onehot[i][Yte[i]] = 1
    return Ytr_onehot, Yte_onehot

In [ ]:
# load the train and test data
a=open(input_x,'r')
Xt = []
while True:
    temp = a.readline()
    if temp=='':break
    else : exec("Xt.append(%s)" %temp)
a.close()

a=open(input_y,'r')
Yt = []
exec("Yt=" +a.readline())
a.close()

Xt = np.array(Xt) - No_data
Yt = np.array(Yt) 
test_idxs = np.random.permutation(len(Xt))
Xt = Xt[test_idxs]
Yt = Yt[test_idxs]

#Train/Valid Sep
TVS = 1200
Xtr, Ytr, Xte, Yte = Xt[:TVS],Yt[:TVS],Xt[TVS:],Yt[TVS:]


# label data of train and test data, label data is represented by one-hot encoding
Ytr_onehot, Yte_onehot = onehot_encoding(Ytr, Yte)


In [ ]:
L0 = tf.placeholder(tf.float32, [None, int(image_x), int(image_y), int(image_z)])
Y = tf.placeholder(tf.float32, [None, int(y_class)])
dropout_prob = tf.placeholder(tf.float32)

In [ ]:
last_layer = 0
image_size = int(image_x)
last_cha = int(image_z)
def max_p(n):
    exec("global L%s \nL%s = tf.nn.max_pool(L%s, ksize=[1,n,n,1],strides=[1,n,n,1,],padding='SAME')" %(last_layer,last_layer,last_layer))
    global image_size
    image_size = round(image_size/n)
    
def conv(n, cha_from, cha_to, res=0, relu=1, p=0): 
    # n:layer번호 / cha_from: 이전 채널 수 / cha_to: 새 채널 수 / res: 양수-only add / relu: relu / p: pool
    #exec("global L%s" %n)
    global last_cha, last_layer
    last_cha =(cha_to)
    last_layer = (n)
    exec("global W%s \nW%s = tf.Variable(tf.random_normal([3,3,cha_from,cha_to],stddev=0.01))" %(n,n))
    if res == 0:
        exec("global L%s \nL%s = tf.nn.conv2d(L%s,W%s,strides=[1,1,1,1],padding='SAME')" %(n,n,n-1,n))
        if relu == 1:
            exec("global L%s \nL%s = tf.nn.relu(L%s)" %(n,n,n))
            exec("global L%s \nL%s = tf.contrib.layers.batch_norm(L%s)" %(n,n,n))

    else:
        if res > 0:
            exec("global L%s \nL%s = tf.add(L%s,L%s)"   %(n,n,n-res,n-1))
            if relu == 1:
                exec("global L%s \nL%s = tf.nn.relu(L%s)" %(n,n,n))
                exec("global L%s \nL%s = tf.contrib.layers.batch_norm(L%s)" %(n,n,n))
        else:   
            exec("global L%s \nL%s = tf.nn.conv2d(L%s,W%s,strides=[1,1,1,1],padding='SAME')" %(n,n,n+res,n))
            exec("global L%s \ntf.add(L%s,L%s)"   %(n,n,n-1))
            if relu == 1:
                exec("global L%s \nL%s = tf.nn.relu(L%s)" %(n,n,n))
                exec("global L%s \nL%s = tf.contrib.layers.batch_norm(L%s)" %(n,n,n))
                
def full(n,che_from,che_to):
    global last_cha, last_layer,image_size
    last_cha =(che_to)
    last_layer = (n)
    
    exec("global W%s \nW%s = tf.Variable(tf.random_normal([%s,%s],stddev=0.01))" %(n,n,int(image_size*image_size*che_from),int(che_to)))
    exec("global L%s \nL%s = tf.reshape(L%s,[-1,%s])" %(n,n,n-1,(int(image_size*image_size*che_from))))
    exec("global B%s \nB%s =  tf.Variable(tf.random_normal([%s],stddev=0.01))" %(n,n,int(che_to)))
    exec("global L%s \nL%s =tf.matmul(L%s,W%s) + B%s " %(n,n,n,n,n))
    exec("L%s = tf.nn.relu(L%s)" %(n,n))
    image_size = 1

In [ ]:
### Implement the layers of CNNs ###


exec(structure)
exec("W%s = tf.Variable(tf.random_normal([%s,%s],stddev=0.01))" %(last_layer+1,int(last_cha),y_class))
exec("model = tf.matmul(L%s,W%s)" %(last_layer,last_layer+1))

In [ ]:
# cost function, you can change the implementation
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(Jump).minimize(cost)

In [ ]:
# initialize the variables
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


In [ ]:
def graph():
    
    img = Image.open('/home/snu/Downloads/blank.png')
    draw_img = ImageDraw.Draw(img)

    x = 0
    acc_min = min(acc_list) ; acc_max = max(acc_list)
    cost_min = min(cost_list); cost_max = max(cost_list)
    
    draw_img.text(xy=(100,70),text=str('ACCURACY'),fill=(0,0,255,255))
    draw_img.text(xy=(100,370),text=str('COST'),fill=(0,0,255,255))
    for i in range(len(epoch_list)):
        x = x + 30 
        y = 300 - (float((acc_list[i]-acc_min)/(acc_max-acc_min))*200 )
        draw_img.line((x,300,x,y), width=10, fill=(0,0,255,255))
        draw_img.text(xy=(x-6,310),text=str(epoch_list[i]),fill=(255,0,0,255))
        draw_img.text(xy=(x-10,320),text=str(round(acc_list[i],2)),fill=(0,0,255,255))
        
        y = 600 - (float((cost_list[i]-cost_min)/(cost_max-cost_min))*200 )
        draw_img.line((x,600,x,y), width=10, fill=(0,0,255,255))
        draw_img.text(xy=(x-6,610),text=str(epoch_list[i]),fill=(255,0,0,255))
        draw_img.text(xy=(x-10,620),text=str(round(cost_list[i],2)),fill=(0,0,255,255))

    img.save('/home/snu/Downloads/blank2.png')

In [ ]:
best_acc = 0
epoch = 0
acc_list= []
epoch_list = []
cost_list = []
def accu(sub_info):
    global accuracy
    test_batch_size = int(Batch_Y)
    test_datasize,_,_,_ = Xte.shape
    test_total_batch = int(test_datasize / test_batch_size)

    test_idxs = np.random.permutation(test_datasize)
    Xte_random = Xte[test_idxs]
    Yte_random = Yte_onehot[test_idxs]

    correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
    partial_accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))

    accuracy = 0
    #print('Partial Accuracies')
    for i in range(test_total_batch):
        t_batch_X = Xte_random[i * test_batch_size:(i+1) * test_batch_size].reshape(-1,int(image_x),int(image_y),int(image_z))
        t_batch_Y = Yte_random[i * test_batch_size:(i+1) * test_batch_size]

        p_acc = sess.run(partial_accuracy,
            feed_dict={
            L0: t_batch_X.reshape(-1, int(image_x), int(image_y), int(image_z)),
            Y: t_batch_Y,
            dropout_prob: 1})

        #print(p_acc)
        accuracy += p_acc

    accuracy = accuracy / test_total_batch
    print('*** Accuracy : {:,.4f} ***'.format(accuracy))
    
    epoch_list.append(epoch); acc_list.append(accuracy); cost_list.append(avg_cost)
    if len(epoch_list) > 20:
        epoch_list.pop(0); acc_list.pop(0) ; cost_list.pop(0)
    try:graph()
    except:pass
    
    
    global best_acc
    if accuracy >= 0.75 and accuracy >= best_acc:
        best_acc =  accuracy
        email_sending.email2me('New Record! '+str(best_acc), sub_info+'\n'+structure, '/home/snu/Downloads/blank2.png')
        email_sending.email2u('New Record! '+str(best_acc), sub_info+'\n'+structure, '/home/snu/Downloads/blank2.png')

In [ ]:
### Implement the train process ###
batch_size= int(Batch_X)
total_batch = int(len(Xtr)/batch_size)

off_switch = 0
while True:
    
    epoch += 1
    total_cost = 0 
    for i in range(total_batch):
        batch_xs,batch_ys = Xtr[i*batch_size: (i+1)*batch_size],Ytr_onehot[i*batch_size: (i+1)*batch_size]
        batch_xs = batch_xs.reshape(-1,int(image_x),int(image_y),int(image_z))
            
        _,curr_loss = sess.run([optimizer,cost],   
                              feed_dict ={L0:batch_xs,Y:batch_ys,dropout_prob:0.7})
        total_cost += curr_loss
    
    avg_cost = total_cost/total_batch
    print('EPOCH : ' , '%04d' % (epoch),
         'Avg.cost = ', '{:,.3f}'.format(avg_cost))
  


    
    
    accu('EPOCH: '+str(epoch)+' Avg.Cost: '+str(avg_cost))
    if best_acc > accuracy:
        off_switch += 1
        if off_switch == 19: 
            email_sending.email2me(input_x +'Learning Finished with '+str(best_acc),'Last EPOCH: '+str(epoch)+' Avg.Cost: '+str(total_cost/total_batch)+'\n'+structure,'/home/snu/Downloads/blank2.png') 
            email_sending.email2u(input_x +'Learning Finished with '+str(best_acc),'Last EPOCH: '+str(epoch)+' Avg.Cost: '+str(total_cost/total_batch)+'\n'+structure,'/home/snu/Downloads/blank2.png')
            break
    elif best_acc <= accuracy: off_switch = 0 
    


# Ensemble

# initialize
sess = tf.Session()

models = []
num_models = 10
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.global_variables_initializer())

print('Learning Started!')

# train my model
for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        # train each model
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

print('Learning Finished!')

# Test model and check accuracy
test_size = len(mnist.test.labels)
predictions = np.zeros([test_size, 10])
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(
        mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p

ensemble_correct_prediction = tf.equal(
    tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(
    tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))
